In [ ]:
import torch

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn

import time
import os
import random

from utils import *
from dataset import *
from encoder import Encoder
from decoder import DecodeNext, Decoder
from predictor import Predictor

from rdkit.Chem import MolFromSmiles, MolToSmiles

from IPython.display import Image
%load_ext autoreload
%autoreload 2

In [ ]:
DATADIR = './data/gdb13/'
OUTDIR = './run11/'
DATASPEC_FILE = os.path.join(DATADIR, 'spec.json')

dataspec = fetch_params(DATASPEC_FILE)

In [ ]:
L = 128

if 'run9' in OUTDIR:
    encoder = torch.load(OUTDIR + 'encoder.pth')
    decoder = torch.load(OUTDIR + 'decoder.pth')
    decoder.smile_len = dataspec.smile_len
    decoder.alphabet = dataspec.alphabet
    decoder.stoi = dataspec.stoi
    predictor = torch.load(OUTDIR + 'predictor.pth')
else:
    encoder = Encoder(L, dataspec)
    encoder.load_state_dict(torch.load(OUTDIR + 'encoder_weights.pth', map_location=torch.device('cpu')))

    decoder = Decoder(L, dataspec)
    decoder.load_state_dict(torch.load(OUTDIR + 'decoder_weights.pth', map_location=torch.device('cpu')))

    predictor = Predictor(L)
    predictor.load_state_dict(torch.load(OUTDIR + 'predictor_weights.pth', map_location=torch.device('cpu')))


In [ ]:
trainset, testset = make_data('./data/gdb13', n=1000)

In [ ]:
means, stds = get_latent_distributions(encoder, testset.hots, plot=True)

In [ ]:
print('Variant Dimensions: ', [i for i, std in enumerate(stds) if std > 0.2])

In [ ]:
CE_LOSS = lambda x, x_hat: float(torch.nn.functional.cross_entropy(x_hat.transpose(1, 2), torch.argmax(x, dim=2)))

n = 100

sample = trainset.sample(n)

losses = [0] * L 

for i, x in enumerate(sample):
    x = x.unsqueeze(0)
    
    mu, _, _ = encoder(x)

    reference_loss = CE_LOSS(x, decoder(mu))

    for i in range(L):
        weight = float(mu[:, i])
        mu[:, i] = 0
        losses[i] += CE_LOSS(x, decoder(mu)) - reference_loss
        mu[:, i] = weight

losses = [loss/n for loss in losses]

In [ ]:
print('Sensitive Dimensions: ', [i for i, loss in enumerate(losses) if loss > 0.05])

In [ ]:
fig, ax = plt.subplots()
ax.bar(range(L), losses)
ax.set_xlabel('dimension number')
ax.set_ylabel('loss difference')
ax.set_title('change in decoder loss from zeroing each latent dimension (run9, 100 samples)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.show()

In [ ]:
smiles = set()
invalid = 0

n = 1000

for i in range(n):
    z = torch.normal(means, stds).unsqueeze(0)
    x_hat = decoder(z)
    smile = from_hot(x_hat, dataspec.alphabet)[0].replace('L', 'Cl')
    
    mol = MolFromSmiles(smile)
    
    if mol:
        smile = MolToSmiles(mol)
        smiles.add(smile)
    else:
        invalid += 1

In [ ]:
print('Diversity:\t', len(smiles)/(n - invalid))
print('Percent Valid:\t', 1 - invalid/n)

In [ ]:
predictor.eval()

z, _, _ = encoder(testset.hots[1:2])

z = z.squeeze()

dim = 124

#interval = torch.randn(1000) * stds[dim] + means[dim]

logps = []
qeds = []
sass = []

mean = float(means[dim])
std = float(stds[dim])

interval = torch.arange(start=mean-3*std, end=mean+3*std, step=2*std/1000)

for d in interval:
    z[dim] = d
    logp, qed, sas = predictor(z.unsqueeze(0))
    logps.append(float(logp))
    qeds.append(float(qed))
    sass.append(float(sas))

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, sharex=True)
ax0.plot(interval, logps)
ax1.plot(interval, qeds)
ax2.plot(interval, sass)